In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

### 제주도 농어촌민박(펜션, 게스트하우스, 민박 등)
- 출처 - 제주데이터허브
- https://jejudatahub.net/data/view/data/956


In [4]:
df_accomo = pd.read_csv('./data/제주농어촌민박현황.csv')
df_accomo.head(20)

,읍면동,민박명,소재지,데이터기준일
0,한림읍,기영 민박,제주시 한림읍 한림로 345,20211123
1,한림읍,에너벨리,제주시 한림읍 한림로 356,20211123
2,한림읍,협재 빌리지,제주시 한림읍 한림로 338-1,20211123
3,한림읍,해송민박,제주시 한림읍 협재4길 7-15,20211123
4,한림읍,영심 민박,제주시 한림읍 협재4길 7-7,20211123
5,한림읍,원룸민박,제주시 한림읍 비양도길 275,20211123
6,한림읍,바다그리기,제주시 한림읍 옹포2길 16,20211123
7,한림읍,비양도 민박,제주시 한림읍 비양도길 8,20211123
8,한림읍,해조대 민박,제주시 한림읍 한림로 339,20211123
9,한림읍,이쁜새민박,제주시 한림읍 협재로 68-5,20211123


In [5]:
accomo_list = df_accomo['민박명']
accomo_list

0              기영 민박
1               에너벨리
2             협재 빌리지
3               해송민박
4              영심 민박
            ...     
2979         화랑펜션2층집
2980    김군빌리지 게스트하우스
2981           도두봉팬션
2982           바다성펜션
2983             올레뷰
Name: 민박명, Length: 2984, dtype: object

In [12]:
accomo_list1 = df_accomo[df_accomo['읍면동'] == '한림읍']['민박명']
accomo_list1

0       기영 민박
1        에너벨리
2      협재 빌리지
3        해송민박
4       영심 민박
        ...  
459    포인트 무드
460    스테이 담담
461        오닐
462     호미하우스
463       코티지
Name: 민박명, Length: 464, dtype: object

### 네이버 블로그 데이터 확인



In [19]:
# 제주 + "제주낭만펜션" 검색결과 URL
# "" 포함시 필수검색 옵션
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+%22%EB%82%AD%EB%A7%8C%EC%A0%9C%EC%A3%BC%ED%8E%9C%EC%85%98%22+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C&tqi'

In [18]:
# 122번까지
result = []
cnt = 0

for accomo in tqdm(accomo_list):

    # 숙소 이름별 - 검색
    base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
    accomo_name = quote(f'"{accomo}"')

    # 제외하고자 하는 검색어
    # 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
    # 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
    delete_words = ["파트너스", "예약코드"]
    delete_words

    delete_search = ""
    for word in delete_words:    
        delete_search += f'+-{quote(word)}'

    # 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
    url = base_url + accomo_name + delete_search
    driver = webdriver.Chrome()
    driver.get(url)


    # 스크롤 내리기

    # len(soup.select('.detail_box')) = 30 첫페이지에 나오는 수가 30개
    # 30개 이하인 경우 분석에서 제외
    if len(driver.find_elements(By.CLASS_NAME, 'detail_box')) > 30:
        continue
    
    else :
        # 스크롤은 3번만 내리자
        for _ in range(3):
            last_height = driver.execute_script("return document.body.scrollHeight")

            # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
            driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
            time.sleep(round(random.uniform(0.0, 1.0), 3))
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
            time.sleep(round(random.uniform(0.0, 1.2), 3))

            # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
    
    time.sleep(round(random.uniform(0.0, 2.2), 3))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select_one('.lst_view').select('.view_wrap')
    
    title_list = []
    content_list = []
    blog_list = []

    # 블로그가 10개 미만인 경우 분석에서 제외
    if len(lis) >= 10:
        for li in lis:
            try :
                title_raw = li.select_one('.title_area').select_one('a').text.strip()
                #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
                title_list.append(title_raw)
            except:
                title_list = 'error'
            
            try :
                content_raw = li.select_one('.dsc_link').text.strip()
                #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
                content_list.append(content_raw)  
            except :
                content_list = 'error'
            try :          
                blog_link = li.select_one('.title_area').select_one('a')['href']
                blog_list.append(blog_link)
            except :
                blog_list = 'error'


    result.append({'name' : accomo,
                   'title' : title_list,
                   'contents' : content_list,
                   'blog' : blog_list})
    cnt += 1
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    print(f'{cnt}번째 "{accomo}" 블로그 링크 수집완료')
time.sleep(round(random.uniform(0.0, 4.2), 3))



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:09<01:26,  9.56s/it]

1번째 "기영 민박" 블로그 링크 수집완료


 20%|██        | 2/10 [00:18<01:11,  8.96s/it]

2번째 "에너벨리" 블로그 링크 수집완료


...

8번째 "비양도 민박" 블로그 링크 수집완료


 90%|█████████ | 9/10 [01:12<00:08,  8.25s/it]

9번째 "해조대 민박" 블로그 링크 수집완료


100%|██████████| 10/10 [01:19<00:00,  7.94s/it]

10번째 "이쁜새민박" 블로그 링크 수집완료


In [36]:
df1 = pd.DataFrame(result)
df1['blog_num'] = df1['blog'].apply(lambda x: len(x))

df1.tail(20)

,name,title,contents,blog,blog_num
102,푸루다비치 오션,[],[],[],0
103,카사블루 민박,[],[],[],0
104,하르방 산장 민박,[],[],[],0
105,풍경 소리 민박,[],[],[],0
106,제주 마중 민박,[],[],[],0
107,제주 하늘 바다 민박,[],[],[],0
108,바르 민박,[],[],[],0
109,쿠잉썸 민박,[],[],[],0
110,똘똘이 민박,[],[],[],0
111,곱드락 펜션민박,[],[],[],0


In [32]:
df1['title'] =df1['title'].apply(lambda x: ' '.join(x))
df1['contents'] =df1['contents'].apply(lambda x: ' '.join(x))
df1

,name,title,contents,blog,blog_num
0,기영 민박,[제주여행] 제주 마지막날 Day3 / 바다에서 하염없이 멍때리기... 제주 혼자 ...,옮겨 #기영민박 들려서 맥주 병나발 불고 육지로 돌아가기 싫음을 온몸으로 표현 돈벌...,"[https://blog.naver.com/imvaney/223121121422, ...",90
1,에너벨리,"해수욕장(동쪽주차장), 협재 에너벨리(식당), 카페노티드제주 몰아 올리는 제주 여행...",제주 #협재에너벨리 - 주소 : 제주 제주시 한림읍 한림로 356 에너벨리식당&민박...,[https://blog.naver.com/hi_7312319/22285822760...,120
2,협재 빌리지,2021.11.07(일)트랜스제주50 후 백패킹 금능해수욕장 제주협재해수욕장펜션 협...,옴기고 협재빌리지모텔민박 064- 796-4165 5만원에 방잡음 덴트 철수하러 토...,"[https://blog.naver.com/mowcho/222563341734, h...",15
3,해송민박,[19.03.02 ~ 05] DAY3~6 - 퇴사 여행. 8박 9일 제주 자유여행 ...,저녁 먹으러 주변에 나가볼까 했는데 죄다 브레이크타임 걸려서 포기 바람 개오졌던 협...,"[https://blog.naver.com/e__uni_/221542178596, ...",18
